In [10]:
from json import dumps
from typing import Callable
import inspect

class Persistence:
    def write(msg: str):
        raise NotImplementedError("Not implemented")
    
class StdoutPersistence(Persistence):
    def write(self, msg):
        print(msg)

class FilePersistence(Persistence):
    def __init__(self, file_name: str):
        super().__init__()
        self._file_name = file_name

    def write(self, msg):
        with open(self._file_name, 'a') as d:
            d.write(msg + "\n")

stdout_persistence = StdoutPersistence()
file_persistence = FilePersistence('/tmp/python-035.log')

def logger(persistence: Persistence):
    def fn_wrapper(fn: Callable):
        signature = inspect.signature(fn)
        fn_name = fn.__name__
        print(f"Decorating {fn_name}")
        for param in signature.parameters.values():
            print(f" Arg: {param.name}, Type: {param.annotation}, Default: {param.default}")

        def args_wrapper(*args, **kwargs):
            fn_name = fn.__name__
            args_str = dumps(args)
            kwargs_str = dumps(kwargs)
            persistence.write(f"Calling function {fn_name} with args: {args_str} and kwargs: {kwargs_str}")
            res = fn(*args, **kwargs)
            res_json = dumps(res)
            persistence.write(f"Return function {fn_name}: {res_json}")
            return res
        return args_wrapper
    return fn_wrapper


@logger(stdout_persistence)
def calcular_suma(sum1: int, sum2: int):
    return sum1 + sum2

calcular_suma(1,2)
calcular_suma(1,3)

@logger(file_persistence)
def calcular_resta(sum1: int, sum2: int):
    return sum1 - sum2

calcular_resta(2,1)
calcular_resta(2,2)

Decorating calcular_suma
 Arg: sum1, Type: <class 'int'>, Default: <class 'inspect._empty'>
 Arg: sum2, Type: <class 'int'>, Default: <class 'inspect._empty'>
Calling function calcular_suma with args: [1, 2] and kwargs: {}
Return function calcular_suma: 3
Calling function calcular_suma with args: [1, 3] and kwargs: {}
Return function calcular_suma: 4
Decorating calcular_resta
 Arg: sum1, Type: <class 'int'>, Default: <class 'inspect._empty'>
 Arg: sum2, Type: <class 'int'>, Default: <class 'inspect._empty'>


0

In [5]:
!cat /tmp/python-035.log

Calling function calcular_resta with args: [2, 1] and kwargs: {}
Return function calcular_resta: 1


In [11]:
from enum import Enum

class TipoPersistencia(Enum):
    TXT = 1
    DB = 2
    HTTP = 3

def super_logger(tipo_persistencia: TipoPersistencia):
    def fn_wrapper(fn: Callable):
        def invocation_wrapper(*args, **kwargs):
            match tipo_persistencia:
                case TipoPersistencia.TXT:
                    print("A txt")
                case TipoPersistencia.DB:
                    print("A db")
                case TipoPersistencia.HTTP:
                    print("A http")
            return fn(*args, **kwargs)
        return invocation_wrapper
    return fn_wrapper

@super_logger(TipoPersistencia.TXT)
def sumar(sum1, sum2):
    return sum1+sum2

sumar(1,2)

        



A txt


3

In [23]:
class UsersRepository:
    def find(query: dict = {}):
        return []



class UsersController:
    def __init__(self, users_repo: UsersRepository, headers: dict):
        self._users_repo = users_repo
        self._headers = headers

    def get_users(self):
        return self._users_repo.find()


In [24]:
!pip install dependency_injector


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [25]:
from dependency_injector import containers, providers
from dependency_injector.wiring import Provide, inject


In [26]:
from dependency_injector import containers, providers

class Container(containers.DeclarativeContainer):
    wiring_config = containers.WiringConfiguration(modules=["__main__"])

    config = providers.Configuration()
    headers = providers.Dependency()  # <--- Define las headers como una dependencia

    users_repo = providers.Singleton(UsersRepository)
    users_controller = providers.Factory(
        UsersController,
        users_repo=users_repo,
        headers=headers,  # <--- Inyecta las headers en el controlador
    )


In [27]:

container = Container()
container.config.from_dict({"some_config": "some_value"})  # <--- Carga la configuración
headers = {"Authorization": "Bearer some_token"}  # <--- Define las headers
container.headers.override(headers)  # <--- Proporciona las headers al contenedor

users_controller = container.users_controller()  # <--- Crea el controlador con las headers inyectadas
users_controller.get_users()

[]